https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset

In [1]:
from google.colab import output
#Install Keras-tuner on first run.
!pip install keras-tuner
# install the ydata-profiling package
!pip install ydata-profiling
#Mount google drive on first run
from google.colab import drive
drive.mount('/content/drive')
#Clear cell output
output.clear() #Clears output after installing packages. Hide if any issues.

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import joblib

#  Import and read the charity_data.csv.
url = "drive/MyDrive/BootCamp/project_4/Resources/healthcare-dataset-stroke-data.csv"
df = pd.read_csv(url)

#Import python auto EDA
# load the ydata_profiling package
from ydata_profiling import ProfileReport

#drop id column
df = df.drop(columns=['id'])



In [3]:
#preliminary changes to format categorical columns and drop id.
#create backup of original df
df_backup = df.copy()
# List of feature names to convert to categorical
categorical_features = ['gender', 'hypertension', 'heart_disease', 'ever_married', 'smoking_status', 'stroke', 'work_type', 'Residence_type']

# Convert the selected features to categorical
df[categorical_features] = df[categorical_features].astype('category')



#Create AutoEDA report

# Cleaning Data

Done in stroke_data_EDA.ipynb

# Data Cleaning

In [4]:
#Need to deal with NaN BMI
#creating new DF with values replaced with mean
df_clean = df.copy()
df_clean = df_clean.drop(columns = ['bmi'])

Encode labels with label encoder.

In [12]:
#set encoder
encoder = LabelEncoder()

#create copy of dataset for encoding to leave for other methods.
df_clean_encoded = df_clean.copy()

# Features to apply label encoding to
features_to_encode = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

# Initialize the LabelEncoder
encoder = LabelEncoder()

# Loop through the features and apply label encoding
for feature in features_to_encode:
    df_clean_encoded[feature] = encoder.fit_transform(df_clean_encoded[feature])

# Display the modified DataFrame
df_clean_encoded.head()


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,228.69,36.600000,1,1
1,0,61.0,0,0,1,3,0,202.21,28.893237,2,1
2,1,80.0,0,1,1,2,0,105.92,32.500000,2,1
3,0,49.0,0,0,1,2,1,171.23,34.400000,3,1
4,0,79.0,1,0,1,3,0,174.12,24.000000,2,1


In [13]:
#Create target and datset from cleaned data
y = y=df_clean_encoded["stroke"]
X=df_clean_encoded.drop(["stroke"], axis=1)

In [14]:
#Scale features with standard scaler
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)

In [15]:
#split model
X_train,X_test,y_train,y_test=train_test_split(X_scaled, y, test_size=0.2,random_state=45)

In [16]:
from sklearn.ensemble import RandomForestClassifier
RandomForestClassifier = RandomForestClassifier(random_state=40, n_jobs=-1, n_estimators=10)
# Training a Classifier
RandomForestClassifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=10, n_jobs=-1, random_state=40)

In [17]:
predict = RandomForestClassifier.predict(X_test)
print(classification_report(y_test, predict))


              precision    recall  f1-score   support

           0       0.96      0.99      0.97       976
           1       0.17      0.02      0.04        46

    accuracy                           0.95      1022
   macro avg       0.56      0.51      0.51      1022
weighted avg       0.92      0.95      0.93      1022



#Hyperparameter tuning attempt on RandomForest

In [21]:
# Define the hyperparameter grid
param_grid = {
    'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 140, 160, 180, 200],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Create a RandomForestClassifier object
rf_classifier = RandomForestClassifier(random_state=40, n_jobs=-1)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf_classifier,
    param_distributions=param_grid,
    n_iter=10,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    random_state=42
)

# Perform hyperparameter tuning on your data (X_train, y_train)
random_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = random_search.best_params_
print("Best Hyperparameters:", best_params)

# Get the best model
best_rf_classifier = random_search.best_estimator_

Best Hyperparameters: {'n_estimators': 60, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 10, 'bootstrap': False}


In [22]:
predict = best_rf_classifier.predict(X_test)
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

           0       0.95      1.00      0.98       976
           1       0.00      0.00      0.00        46

    accuracy                           0.95      1022
   macro avg       0.48      0.50      0.49      1022
weighted avg       0.91      0.95      0.93      1022



In [24]:
# Define the path to the Google Drive directory where you want to save your model
drive_path = '/content/drive/MyDrive/BootCamp/project_4/'  # Update "Your_Folder" with your desired folder name

# Save the model to the specified path
model_filename = 'stroke_data_decision_tree_model.pkl'  # Replace "your_model_name" with your model's name and extension
joblib.dump(best_rf_classifier, drive_path + model_filename)

['/content/drive/MyDrive/BootCamp/project_4/stroke_data_decision_tree_model.pkl']